In [ ]:
from http.server import BaseHTTPRequestHandler, ThreadingHTTPServer
from _thread import start_new_thread
from myHttp import http  # pip install myHttp
from myBasics import base64ToBin, binToBase64  # pip install myBasics
import logging
from typing import Any
import json
from time import sleep
from threading import Lock
BASE_URL = 'https://www.wuyunai.com'
DOMAIN = 'www.wuyunai.com'
# IP_PORT='192.168.1.142:9005'
IP_PORT = '127.0.0.1:9005'

In [ ]:
try:
    f = open('cache.json')
    cache = f.read()
    f.close()
    cache = json.loads(cache)
except:
    cache = {}
f = open('./autojspro.log', 'w')
lock = Lock()


def safe_write(a):
    with lock:
        f.write(a + '\n')
        f.flush()


def to_dict(header):
    # for self.headers
    d = {}
    for k in header:
        d[k] = header[k]
    return d


class Resquest(BaseHTTPRequestHandler):
    def do_GET(self):
        safe_write(f"{self.path} request received")
        url = self.path
        h = to_dict(self.headers)
        h['Host'] = DOMAIN
        if (url in cache):
            status = cache[url][0]
            content = cache[url][1]
            header = cache[url][2]
            self.send_response(status)
            self.end_headers()
            self.wfile.write(base64ToBin(content))
            safe_write(url + ' from cache')
            return
        r = http(BASE_URL + url, Header=h, Timeout=10000, Decode=False)
        assert (r['status'] == 0)
        safe_write(url + ' cached')
        status = r['code']
        content = r['text']
        header = r['header']
        for k in header:
            header[k] = header[k].replace(DOMAIN, IP_PORT)
        header['Content-Length'] = str(len(content))
        cache[url] = [status, binToBase64(content), header]
        self.send_response(status)
        self.end_headers()
        self.wfile.write(content)

    def log_message(self, format: str, *args: Any) -> None:
        pass

In [ ]:
def save():
    text = json.dumps(cache, ensure_ascii=False)
    with open('cache.json', 'w') as f:
        f.write(text)

In [ ]:
print(len(cache))

In [ ]:
server = ThreadingHTTPServer(('0.0.0.0', 9005), Resquest)
start_new_thread(server.serve_forever, ())

In [ ]:
while True:
    sleep(10)
    safe_write('saving ...')
    save()
    safe_write('saved')